In [8]:
import pandas as pd
import numpy as np

### data from  https://archive.ics.uci.edu/ml/datasets/Electrical+Grid+Stability+Simulated+Data+#

In [9]:
df = pd.read_csv("data/Data_for_UCI_named.csv")
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [10]:
df = df.drop('stab', axis=1)

In [11]:
X = df.drop('stabf', axis=1)
y = df['stabf']

In [24]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [13]:
scaler = StandardScaler()

In [14]:
Xt = scaler.fit_transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(Xt, y, test_size=0.2, random_state=1)

In [3]:
import xgboost as xgb

##### Question

In [38]:
etc = ExtraTreesClassifier()

In [39]:
etc.fit(X_train, y_train)

ExtraTreesClassifier()

In [40]:
accuracy_score(y_test, etc.predict(X_test))

0.9295

In [42]:
etc2 = ExtraTreesClassifier(random_state=1)

In [43]:
etc2.fit(X_train, y_train)

ExtraTreesClassifier(random_state=1)

In [44]:
accuracy_score(y_test, etc2.predict(X_test))

0.928

##### Question

In [60]:
for i in zip(etc.feature_importances_, df.columns):
    print(i)

(0.11662181555075143, 'tau1')
(0.11847486818270028, 'tau2')
(0.11486136365707982, 'tau3')
(0.11631534200634519, 'tau4')
(0.03925026037121737, 'p1')
(0.0403113519457329, 'p2')
(0.04008875776118417, 'p3')
(0.04054008344096217, 'p4')
(0.08939071217498631, 'g1')
(0.09486222103357866, 'g2')
(0.09680807990271, 'g3')
(0.09247514397275172, 'g4')


##### Question

In [47]:
param_grid = {
    "n_estimators": [100, 300, 500, 1000],
    "min_samples_split": [2,  7, 5],
    "min_samples_leaf": [4, 8, 6],
    "max_features": [None, 'log2', 'auto'],
}

grid = GridSearchCV(etc,
                   param_grid, n_jobs=-1, cv=5)

In [ ]:
grid.fit(X_train, y_train)

##### Question

In [33]:
rfc = RandomForestClassifier(random_state=1)

In [35]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(y_test, rfc.predict(X_test))

0.929

##### Question

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
# specify parameters via map
param = {'objective':'binary:logistic'}
bst = xgb.train(param, dtrain)
# make prediction
preds = bst.predict(dtest)